In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, WeightedRandomSampler
import numpy as np
from tqdm import tqdm
from torchvision.models.efficientnet import EfficientNet_B2_Weights

# Paths
data_dir = "../FBMM/test"
train_dir = os.path.join(data_dir, "train")
val_dir = os.path.join(data_dir, "val")
test_dir = os.path.join(data_dir, "test")
model_save_path = "./models/optimized_efficientnet_b2_emotion_model.pth"

# Configuration
batch_size = 32  # ✅ Reduced for better generalization
num_epochs = 50  # ✅ Increased to 50 with early stopping
initial_lr = 1e-3
weight_decay = 1e-4  # ✅ L2 Regularization
num_classes = 7
img_height, img_width = 260, 260
seed = 42  # For reproducibility
accumulation_steps = 2  # ✅ Gradient accumulation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Emotion categories
emotion_classes = ["Anger", "Disgust", "Fear", "Happy", "Neutral", "Sad", "Surprise"]

# Data Augmentation & Normalization
weights = EfficientNet_B2_Weights.IMAGENET1K_V1
transform = transforms.Compose([
    transforms.Resize((img_height, img_width)),
    transforms.ToTensor(),
    weights.transforms()
])

# Load Datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
val_dataset = datasets.ImageFolder(root=val_dir, transform=transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)

# Compute Class Weights
def compute_class_weights(dataset, num_classes):
    labels = np.array([label for _, label in dataset.samples])
    class_counts = np.bincount(labels, minlength=num_classes)
    class_weights = 1.0 / (class_counts + 1e-6)
    class_weights /= class_weights.sum()
    return torch.tensor(class_weights, dtype=torch.float32).to(device)

class_weights = compute_class_weights(train_dataset, num_classes)

# Data Loaders with Optimized Multi-Processing
def get_sampler(dataset):
    labels = np.array([label for _, label in dataset.samples])
    class_sample_counts = np.bincount(labels)
    weights = 1.0 / (class_sample_counts[labels] + 1e-6)
    return WeightedRandomSampler(weights, len(weights))

train_loader = DataLoader(
    train_dataset, batch_size=batch_size, sampler=get_sampler(train_dataset),
    num_workers=0, pin_memory=True  # 🚀 Fix for Windows multiprocessing issue
)

val_loader = DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False,
    num_workers=0, pin_memory=True
)

test_loader = DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False,
    num_workers=0, pin_memory=True
)

# Load Pretrained EfficientNet Model & Fine-Tuning
def load_model(num_classes):
    print("Loading and configuring the model...")
    
    model = models.efficientnet_b2(weights=weights)  # ✅ Using ImageNet weights
    model = model.to(memory_format=torch.channels_last)  # ✅ Optimize memory format for better GPU efficiency

    # Freeze all layers initially
    for param in model.parameters():
        param.requires_grad = False

    # Unlock last 20% of convolutional layers + classifier head
    total_layers = len(list(model.features.children()))
    fine_tune_layers = int(total_layers * 0.2)

    for layer in list(model.features.children())[-fine_tune_layers:]:
        for param in layer.parameters():
            param.requires_grad = True

    # Modify classifier head
    model.classifier = nn.Sequential(
        nn.Dropout(0.6),
        nn.Linear(model.classifier[1].in_features, num_classes)
    )

    # Ensure classifier is trainable
    for param in model.classifier.parameters():
        param.requires_grad = True

    return model.to(device)

# Load model
model = load_model(num_classes)

# Loss & Optimizer
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=initial_lr, weight_decay=weight_decay)  # ✅ AdamW with L2 regularization

# ✅ Learning Rate Warm-Up & Scheduling
scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=initial_lr, epochs=num_epochs, steps_per_epoch=len(train_loader), pct_start=0.1)

# ✅ Enable Mixed Precision Training
scaler = torch.amp.GradScaler(device="cuda")

# Training Loop with Early Stopping & Optimizations
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs, patience=7):
    best_val_loss = np.inf
    epochs_no_improve = 0

    for epoch in range(1, num_epochs + 1):
        model.train()
        running_loss, correct, total = 0.0, 0, 0

        for i, (images, labels) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch}/{num_epochs}")):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()

            with torch.cuda.amp.autocast():  # ✅ Mixed Precision Training
                outputs = model(images)
                loss = criterion(outputs, labels) / accumulation_steps  # ✅ Gradient Accumulation

            scaler.scale(loss).backward()

            if (i + 1) % accumulation_steps == 0:  # ✅ Update weights after accumulation steps
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
                scheduler.step()  # ✅ Apply LR warm-up & decay

            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        train_loss = running_loss / total
        train_acc = correct / total

        # Validation
        model.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * images.size(0)
                _, predicted = torch.max(outputs, 1)
                val_correct += (predicted == labels).sum().item()
                val_total += labels.size(0)

        val_loss /= val_total
        val_acc = val_correct / val_total

        print(f"\nEpoch {epoch}: Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

        # Early Stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), model_save_path)
            print("Model Saved!")
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                print("Early stopping triggered!")
                break

# Train the model
train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs)

# Evaluation on Test Set
def evaluate_model(model, test_loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    accuracy = correct / total
    print(f"Test Accuracy: {accuracy:.4f}")

# Load best model for testing
model.load_state_dict(torch.load(model_save_path))
evaluate_model(model, test_loader)

Loading and configuring the model...


C:\Users\Tuf\AppData\Local\Temp\ipykernel_23576\1351113480.py:121: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
Epoch 1/50:   0%|          | 0/5 [00:00<?, ?it/s]C:\Users\Tuf\AppData\Local\Temp\ipykernel_23576\1351113480.py:137: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():  # ✅ Mixed Precision Training
Epoch 1/50: 100%|██████████| 5/5 [00:06<00:00,  1.34s/it]



Epoch 1: Train Loss: 0.9867, Train Acc: 0.1286 | Val Loss: 1.9407, Val Acc: 0.1500
Model Saved!


Epoch 2/50: 100%|██████████| 5/5 [00:06<00:00,  1.31s/it]



Epoch 2: Train Loss: 1.0071, Train Acc: 0.1071 | Val Loss: 1.9424, Val Acc: 0.1643


...


Epoch 37: Train Loss: 0.2959, Train Acc: 0.8929 | Val Loss: 1.7959, Val Acc: 0.2714


Epoch 38/50: 100%|██████████| 5/5 [00:06<00:00,  1.29s/it]



Epoch 38: Train Loss: 0.2656, Train Acc: 0.9429 | Val Loss: 1.8027, Val Acc: 0.2929


Epoch 39/50:  20%|██        | 1/5 [00:03<00:12,  3.06s/it]


KeyboardInterrupt: 